<a href="https://colab.research.google.com/github/LeomaxFilho/Operational-Research/blob/main/Pr%C3%A1tica_PLI_BinPacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula prática: Bin Packing


## Exercício 1


### Descrição do problema
Uma faculdade quer agendar as provas de 6 disciplinas. Existe uma regra que proíbe que o mesmo aluno tenha que fazer mais de uma prova por dia. A tabela abaixo mostra quais alunos (identificados por números) vão fazer a prova de cada disciplina. Por exemplo, A e B não podem ser agendadas no mesmo dia por causa do aluno 1. Já B e C poderiam ser agendadas no mesmo dia, pois nenhum aluno vai fazer essas duas provas.

| Disciplina | Alunos |
|:---:|:---:|
| A | 1, 2, 3, 4 |
| B | 1, 5 |
| C | 3, 6, 7 |
| D | 5, 7 |
| E | 2, 7 |
| F | 4, 5 |

Crie um modelo de PLI para agendar essas provas no menor número de dias possível.
**Dica: baseie-se no modelo do problema de empacotamento de caixas.**


### Resolução

In [2]:
# instalação e importação do pacote mip
!pip install mip
from mip import *

# funcões usadas posteriormente:

# resolve o modelo e mostra os valores das variáveis
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  if status != OptimizationStatus.OPTIMAL:
    return

  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
    if v.x > 0.001:
      print(f"{v.name} = {v.x:.2f}")


# salva modelo em arquivo lp, e mostra o conteúdo
def save(model, filename):
  model.write(filename) # salva modelo em arquivo
  with open(filename, "r") as f: # lê e exibe conteúdo do arquivo
    print(f.read())

In [74]:
T = {
    'A' : [1, 2, 3, 4],
    'B' : [1, 5],
    'C' : [3, 6, 7],
    'D' : [5, 7],
    'E' : [2, 7],
    'F' : [4, 5]
    }

A = {}

for i, D in T.items():
  for j in D:
    if j not in A:
      A[j] = []
    A[j].append(i)

print(A)

{1: ['A', 'B'], 2: ['A', 'E'], 3: ['A', 'C'], 4: ['A', 'F'], 5: ['B', 'D', 'F'], 6: ['C'], 7: ['C', 'D', 'E']}


In [75]:
def repeats(T : dict, studant : int, day) -> bool:
  for i in range(day):
    if studant in T[i]:
      return False

In [119]:
model = Model(name='Bin Packing Faculdade', sense=MINIMIZE, solver_name=CBC)
x = [model.add_var(name=f'x_{i}', var_type=BINARY) for i in range(len(T))]

y = {i: [model.add_var(name=f"y_{i}_{j}", var_type=BINARY) for j in range(len(T))] for i in T.keys()}

model.objective = xsum(x[i] for i in range(len(T)))

for j in T.keys():
  model += 1 == xsum(y[j][i] for i in range(len(T)))

for i in A.keys():
  for j in range(len(T)):
    model += x[j] >= xsum(y[z][j]  for z in A[i])

save(model, "model.lp")

\Problem name: Bin Packing Faculdade

Minimize
OBJROW: x_0 + x_1 + x_2 + x_3 + x_4 + x_5
Subject To
constr(0):  y_A_0 + y_A_1 + y_A_2 + y_A_3 + y_A_4 + y_A_5 = 1
constr(1):  y_B_0 + y_B_1 + y_B_2 + y_B_3 + y_B_4 + y_B_5 = 1
constr(2):  y_C_0 + y_C_1 + y_C_2 + y_C_3 + y_C_4 + y_C_5 = 1
constr(3):  y_D_0 + y_D_1 + y_D_2 + y_D_3 + y_D_4 + y_D_5 = 1
constr(4):  y_E_0 + y_E_1 + y_E_2 + y_E_3 + y_E_4 + y_E_5 = 1
constr(5):  y_F_0 + y_F_1 + y_F_2 + y_F_3 + y_F_4 + y_F_5 = 1
constr(6):  x_0 - y_A_0 - y_B_0 >= -0
constr(7):  x_1 - y_A_1 - y_B_1 >= -0
constr(8):  x_2 - y_A_2 - y_B_2 >= -0
constr(9):  x_3 - y_A_3 - y_B_3 >= -0
constr(10):  x_4 - y_A_4 - y_B_4 >= -0
constr(11):  x_5 - y_A_5 - y_B_5 >= -0
constr(12):  x_0 - y_A_0 - y_E_0 >= -0
constr(13):  x_1 - y_A_1 - y_E_1 >= -0
constr(14):  x_2 - y_A_2 - y_E_2 >= -0
constr(15):  x_3 - y_A_3 - y_E_3 >= -0
constr(16):  x_4 - y_A_4 - y_E_4 >= -0
constr(17):  x_5 - y_A_5 - y_E_5 >= -0
constr(18):  x_0 - y_A_0 - y_C_0 >= -0
constr(19):  x_1 - y_A_1 

In [120]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 3.00

Solution:
x_0 = 1.00
x_3 = 1.00
x_4 = 1.00
y_A_3 = 1.00
y_B_4 = 1.00
y_C_4 = 1.00
y_D_3 = 1.00
y_E_0 = 1.00
y_F_0 = 1.00
